In [2]:
import os 
import pandas as pd
import numpy as np

base_dir = "G:/Mon Drive/"

In [3]:
# Cleaning Training CSV - It has not useful labels.
test_img_dir = base_dir + "dataset/test/test"
listimg = [e.split('.')[0] for e in os.listdir(test_img_dir)]
df = pd.DataFrame(listimg)
df.columns = ["img_name"]
df["level"] = 0

In [4]:
from tqdm import tqdm
import cv2

im_size1 = 224
im_size2 = 224

# Lad test data
#df_test = pd.read_csv(base_dir + "dataset/dbfinal_test_upsample.csv")
#df_test.head()
df_test = df
#print(df_test.level.value_counts())

# One hot labels:
targets_series = pd.Series(df_test['level'])
one_hot = pd.get_dummies(targets_series, sparse = True)
one_hot_labels = np.asarray(one_hot)

# Load dataset with filter:
#x_test = []
#y_test = [] 
#i = 0 
#for f, _ in tqdm(df_test.values):
 #   #print(f)
  #  if type(cv2.imread(base_dir+'dataset/test/denoiseCLAHE/{}.jpg'.format(f)))==type(None):
   #     continue
    #else:
     #   img = cv2.imread(base_dir+'dataset/test/denoiseCLAHE/{}.jpg'.format(f))
     #   label = one_hot_labels[i]
      #  x_test.append(cv2.resize(img, (im_size1, im_size2)))
       # y_test.append(label)
        #i += 1
#np.save(base_dir+'dataset/test/x_test_results',x_test)
print('Done')

Done


In [5]:
import tensorflow.keras.backend as K
from tensorflow.keras.losses import CategoricalCrossentropy


class WeightedCategoricalCrossentropy(CategoricalCrossentropy):

    def __init__(self, cost_mat=np.ones((5, 5)), name='weighted_categorical_crossentropy', **kwargs):
        assert(cost_mat.ndim == 2)
        assert(cost_mat.shape[0] == cost_mat.shape[1])

        super().__init__(name=name, **kwargs)
        self.cost_mat = K.cast_to_floatx(cost_mat)

    def __call__(self, y_true, y_pred, sample_weight=None):

        return super().__call__(
            y_true=y_true,
            y_pred=y_pred,
            sample_weight=get_sample_weights(y_true, y_pred, self.cost_mat),
        )
        
    def get_config(self):
        return super().get_config().copy().update(
            {'cost_mat': self.cost_mat}
        )
    #def get_config(self):
    #    return {'cost_mat': self.cost_mat}
        
    @classmethod
    def from_config(cls, config):
       return cls(**config)


def get_sample_weights(y_true, y_pred, cost_m):
    num_classes = len(cost_m)

    y_pred.shape.assert_has_rank(2)
    assert(y_pred.shape[1] == num_classes)
    y_pred.shape.assert_is_compatible_with(y_true.shape)

    y_pred = K.one_hot(K.argmax(y_pred), num_classes)

    y_true_nk1 = K.expand_dims(y_true, 2)
    y_pred_n1k = K.expand_dims(y_pred, 1)
    cost_m_1kk = K.expand_dims(cost_m, 0)

    sample_weights_nkk = cost_m_1kk * y_true_nk1 * y_pred_n1k
    sample_weights_n = K.sum(sample_weights_nkk, axis=[1, 2])

    return sample_weights_n


# Categorical accuracy:
from tensorflow.keras.metrics import CategoricalAccuracy


class WeightedCategoricalAccuracy(CategoricalAccuracy):

    def __init__(self, cost_mat=np.ones((5, 5)), name='weighted_categorical_accuracy', **kwargs):
        assert(cost_mat.ndim == 2)
        assert(cost_mat.shape[0] == cost_mat.shape[1])

        super().__init__(name=name, **kwargs)
        self.cost_mat = K.cast_to_floatx(cost_mat)

    def update_state(self, y_true, y_pred, sample_weight=None):

        return super().update_state(
            y_true=y_true,
            y_pred=y_pred,
            sample_weight=get_sample_weights(y_true, y_pred, self.cost_mat),
        )
#
# model.compile(metrics=[WeightedCategoricalAccuracy(cost_matrix), ...], ...)

# Matriz de costos: 
import numpy as np
w_array = np.ones((5, 5))
# If >1 - it penalize Class i classified as j
i = 0
j = 1 
w_array[i, j] = 1
# Overall enphasis in class i:
w_array[1, :] = 3
w_array[2, :] = 2
w_array[3, :] = 4
w_array[4, :] = 5
cost_mat = w_array


# Load a Model

In [5]:
import tensorflow as tf
#new_model = tf.keras.models.load_model(base_dir + 'models/v_resnet_balancedData/resNet50_p.h5')
new_model = tf.keras.models.load_model(base_dir + 'models/v1_resnet_NOupsample/resNet50_p1.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})
# Check its architecture
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [6]:
import numpy as np
x_test = np.load(base_dir+'dataset/test/x_test_results.npy', mmap_mode='r')


In [7]:
x_test[0]

In [6]:
import numpy as np
x_test = np.load(base_dir+'dataset/test/x_test_results.npy', mmap_mode='r')


# Normalize images:
x_test = np.array(x_test / 255, np.float32)

print(x_test.shape)
#print(y_test.shape)

(10538, 224, 224, 3)


In [7]:
# Predict all validation set:  
y_pred_valid = new_model.predict(x_test, verbose=2)

330/330 - 455s - 455s/epoch - 1s/step


In [10]:
pred = np.argmax(y_pred_valid, axis = 1)
unique, counts = np.unique(pred, return_counts=True)
unique, counts

(array([0], dtype=int64), array([10538], dtype=int64))

## Save predictions

In [ ]:
#$>>> d = {'col1': [0, 1, 2, 3], 'col2': pd.Series([2, 3], index=[2, 3])}
#>>> pd.DataFrame(data=d, index=[0, 1, 2, 3])
data = {'ID':listimg}
df_final = pd.DataFrame(data)
df_final["level"] = pred
df_final
#df_final.to_csv(base_dir + "results/result_v0_naive0.csv", index=False)
df_final.to_csv(base_dir + "results/test_result.csv", index=False)